In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders()

In [12]:
#for x in train_loader:
#    print(x)
#    break

# model

In [13]:
#import pretrainedmodels

In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

In [16]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

In [17]:
class BengaliNet(nn.Module):
    """
    EfficientNet B0-B8.
    Args:
        cfg (CfgNode): configs
    """
    def __init__(self, cfg):
        super(BengaliNet, self).__init__()
        model_name = cfg.MODEL_NAME
        pretrained = cfg.PRETRAINED
        input_channels = cfg.IN_CHANNELS
        pool_type = cfg.POOL_TYPE
        drop_connect_rate = cfg.DROP_CONNECT
        self.drop_rate = cfg.DROPOUT
        cls_head = cfg.CLS_HEAD
        num_total_classes = cfg.NUM_GRAPHEME_CLASSES + cfg.NUM_VOWEL_CLASSES + cfg.NUM_CONSONANT_CLASSES \
            + cfg.NUM_WORD_CLASSES

        backbone = timm.create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=input_channels,
            drop_connect_rate=drop_connect_rate,
        )
        self.conv_stem = backbone.conv_stem
        self.bn1 = backbone.bn1
        self.act1 = backbone.act1
        ### Original blocks ###
        for i in range(len((backbone.blocks))):
            setattr(self, "block{}".format(str(i)), backbone.blocks[i])
        self.conv_head = backbone.conv_head
        self.bn2 = backbone.bn2
        self.act2 = backbone.act2
        self.aux_block5 = backbone.blocks[5]
        self.aux_num_features = self.block5[-1].bn3.num_features
        self.aux_head4 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn4 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act4 = Swish()
        self.aux_head5 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn5 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act5 = Swish()
        self.global_pool = SelectAdaptivePool2d(pool_type=pool_type)
        self.num_features = backbone.num_features * self.global_pool.feat_mult()
        assert cls_head == 'linear'
        if cls_head == "linear":
            ### Baseline head ###
            #self.grapheme_fc = nn.Linear(self.num_features, num_grapheme_classes)
            #self.consonant_fc = nn.Linear(self.num_features, num_consonant_classes)
            #self.vowel_fc = nn.Linear(self.num_features, num_vowel_classes)
            #self.word_fc = nn.Linear(self.num_features, num_word_classes)
            self.fc = nn.Linear(self.num_features, num_total_classes)
            
            #self.grapheme_fc_1 = nn.Linear(self.aux_num_features * 4, num_grapheme_classes)
            #self.consonant_fc_1 = nn.Linear(self.aux_num_features * 4, num_consonant_classes)
            #self.vowel_fc_1 = nn.Linear(self.aux_num_features * 4, num_vowel_classes)
            #self.word_fc_1 = nn.Linear(self.aux_num_features * 4, num_word_classes)
            
            self.aux_fc1 = nn.Linear(self.aux_num_features*4, num_total_classes)
            
            #self.grapheme_fc_2 = nn.Linear(self.aux_num_features * 4, num_grapheme_classes)
            #self.consonant_fc_2 = nn.Linear(self.aux_num_features * 4, num_consonant_classes)
            #self.vowel_fc_2 = nn.Linear(self.aux_num_features * 4, num_vowel_classes)
            #self.word_fc_2 = nn.Linear(self.aux_num_features * 4, num_word_classes)
            
            self.aux_fc2 = nn.Linear(self.aux_num_features*4, num_total_classes)
            
            for fc in [self.fc, self.aux_fc1, self.aux_fc2]:
                nn.init.zeros_(fc.bias.data)
        elif cls_head == "norm_softmax":
            ### NormSoftmax ###
            self.grapheme_fc = NormSoftmax(self.num_features, num_grapheme_classes)
            self.consonant_fc = NormSoftmax(self.num_features, num_consonant_classes)
            self.vowel_fc = NormSoftmax(self.num_features, num_vowel_classes)
        # Replace with Mish activation
        if cfg.MODEL_ACTIVATION == "mish":
            convert_swish_to_mish(self)
        del backbone

    def _features(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x); b4 = x
        x = self.block5(x); b4 = self.aux_block5(b4); b5 = x
        x = self.block6(x)
        x = self.conv_head(x); b4 = self.aux_head4(b4); b5 = self.aux_head5(b5)
        x = self.bn2(x); b4 = self.bn4(b4); b5 = self.bn5(b5)
        x = self.act2(x); b4 = self.act4(b4); b5 = self.act5(b5)
        return b4, b5, x

    def forward(self, x):
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)

        # _, _, x = self._features(x)
        b4, b5, x = self._features(x)
        x = self.global_pool(x); b4 = self.global_pool(b4); b5 = self.global_pool(b5)
        x = torch.flatten(x, 1); b4 = torch.flatten(b4, 1); b5 = torch.flatten(b5, 1)
        if self.drop_rate > 0.:
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        #grapheme_logits = self.grapheme_fc(x)
        #vowel_logits = self.vowel_fc(x)
        #consonant_logits = self.consonant_fc(x)
        #word_logits = self.word_fc(x)
        logits = self.fc(x)
        
        #grapheme_logits_1 = self.grapheme_fc_1(b4)
        #vowel_logits_1 = self.vowel_fc_1(b4)
        #consonant_logits_1 = self.consonant_fc_1(b4)
        #word_logits_1 = self.word_fc_1(b4)
        aux_logits1 = self.aux_fc1(b4)
        
        #grapheme_logits_2 = self.grapheme_fc_2(b5)
        #vowel_logits_2 = self.vowel_fc_2(b5)
        #consonant_logits_2 = self.consonant_fc_2(b5)
        #word_logits_2 = self.word_fc_2(b5)
        aux_logits2 = self.aux_fc2(b5)
        
        #return grapheme_logits, vowel_logits, consonant_logits, word_logits, \
        #       grapheme_logits_1, vowel_logits_1, consonant_logits_1, word_logits_1, \
        #       grapheme_logits_2, vowel_logits_2, consonant_logits_2, word_logits_2
        return logits, aux_logits1, aux_logits2

In [18]:
MODEL_DIR = './model4-ckps'
def create_model(cfg):
    model = BengaliNet(cfg)
    model_file = os.path.join(MODEL_DIR, cfg.MODEL_NAME, cfg.CKP_NAME)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [19]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [20]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [21]:
round(1/9, 6)

0.111111

In [22]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [23]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()

In [24]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, _, _ = model(x)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [25]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [26]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [27]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [28]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [29]:
np.random.random()

0.8841070185469359

In [30]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam

In [31]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [32]:
best_metrics = 0.

def train(args, model):
    optimizer = make_optimizer(model)
    scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        train_cycle(args, model, optimizer, scheduler)
        args.base_lr = 4e-4
        args.num_epochs = 100
        args.warmup_epochs = 10
        scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

def train_cycle(args, model, optimizer, lr_scheduler):
    global best_metrics
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)

    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
    
    model.train()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            batch_size = img.size(0)
            r = np.random.rand()

            if True:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs, outputs_aux1, outputs_aux2 = model(img)
                loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
                loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
                loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
                loss = loss_primary + loss_aux1 + loss_aux2

            optimizer.zero_grad()
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            #loss.backward()
            lr_scheduler(optimizer, batch_idx, epoch)
            optimizer.step()            
            
            current_lr = get_lrs(optimizer)

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

        if True:#train_iter > 0 and train_iter % args.iter_val == 0:
            val_metrics = validate(model, val_loader)
            print('\nval:', val_metrics)
                
            if val_metrics[best_key] > best_metrics:
                best_metrics = val_metrics[best_key]
                save_model(model, model_file)
                print('###>>>>> saved')
                
            model.train()

In [33]:
cfg = Namespace()
cfg.MODEL_NAME = 'tf_efficientnet_b4'
cfg.PRETRAINED = True
cfg.IN_CHANNELS = 1
cfg.POOL_TYPE = 'avg'
cfg.CLS_HEAD = 'linear'
cfg.MODEL_ACTIVATION = 'swish'
cfg.DROP_CONNECT = 0.2
cfg.DROPOUT= 0.
cfg.NUM_WORD_CLASSES = 1295
cfg.NUM_GRAPHEME_CLASSES = 168
cfg.NUM_VOWEL_CLASSES = 11
cfg.NUM_CONSONANT_CLASSES = 7
cfg.CKP_NAME = 'model4_eb4_fold1.pth'

In [34]:
#model, model_file = create_model(cfg)
#model(torch.randn(2,1,137,236))[0].size()

In [35]:
args = Namespace()

args.base_lr = 4e-4
args.num_epochs = 100
args.warmup_epochs = 5
args.num_cycles = 100
args.batch_size = 640
args.val_batch_size = 1024
args.st_epochs = 10

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=1)

(200840, 6)
(200840, 32332)
(160635, 6) (40205, 6)


In [40]:
model, model_file = create_model(cfg)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold1.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold1.pth...


In [ ]:
train(args, model)

CYCLE: 1
{'recall': 0.992515, 'recall_grapheme': 0.990199, 'recall_vowel': 0.995289, 'recall_consonant': 0.994371, 'recall_word': 0.989434, 'acc_grapheme': 0.989379, 'acc_vowel': 0.995249, 'acc_consonant': 0.996219, 'acc_word': 0.989504, 'loss_grapheme': 0.180598, 'loss_vowel': 0.129995, 'loss_consonant': 0.083757, 'loss_word': 0.13174}
    0 | 0.000080 | 160000/160635 | 19.4096 | 15.3317 |
val: {'recall': 0.991266, 'recall_grapheme': 0.987342, 'recall_vowel': 0.9943, 'recall_consonant': 0.99608, 'recall_word': 0.989049, 'acc_grapheme': 0.986693, 'acc_vowel': 0.99423, 'acc_consonant': 0.994976, 'acc_word': 0.989056, 'loss_grapheme': 0.318078, 'loss_vowel': 0.260641, 'loss_consonant': 0.165991, 'loss_word': 0.172336}
    1 | 0.000160 | 160000/160635 | 20.6334 | 14.9313 |
val: {'recall': 0.992183, 'recall_grapheme': 0.988759, 'recall_vowel': 0.994917, 'recall_consonant': 0.996296, 'recall_word': 0.989278, 'acc_grapheme': 0.987862, 'acc_vowel': 0.994777, 'acc_consonant': 0.995821, 'acc_wo

In [38]:
train(args, model)

CYCLE: 1
{'recall': 0.984603, 'recall_grapheme': 0.97713, 'recall_vowel': 0.991312, 'recall_consonant': 0.992841, 'recall_word': 0.97772, 'acc_grapheme': 0.975376, 'acc_vowel': 0.990822, 'acc_consonant': 0.990349, 'acc_word': 0.977764, 'loss_grapheme': 0.377051, 'loss_vowel': 0.251561, 'loss_consonant': 0.166439, 'loss_word': 0.327534}
    0 | 0.000080 | 160000/160635 | 25.9052 | 18.1899 |
val: {'recall': 0.984696, 'recall_grapheme': 0.977376, 'recall_vowel': 0.991461, 'recall_consonant': 0.992571, 'recall_word': 0.978015, 'acc_grapheme': 0.9756, 'acc_vowel': 0.991195, 'acc_consonant': 0.991245, 'acc_word': 0.978038, 'loss_grapheme': 0.340236, 'loss_vowel': 0.240668, 'loss_consonant': 0.161181, 'loss_word': 0.29083}
###>>>>> saved
    1 | 0.000160 | 160000/160635 | 2.3221 | 18.9110 ||
val: {'recall': 0.984781, 'recall_grapheme': 0.978174, 'recall_vowel': 0.991073, 'recall_consonant': 0.991703, 'recall_word': 0.97805, 'acc_grapheme': 0.976694, 'acc_vowel': 0.991319, 'acc_consonant': 0.9

###>>>>> saved
   21 | 0.000354 | 160000/160635 | 15.5916 | 16.6565 |
val: {'recall': 0.98801, 'recall_grapheme': 0.983662, 'recall_vowel': 0.992418, 'recall_consonant': 0.992298, 'recall_word': 0.982332, 'acc_grapheme': 0.981445, 'acc_vowel': 0.992812, 'acc_consonant': 0.992886, 'acc_word': 0.982365, 'loss_grapheme': 0.324178, 'loss_vowel': 0.234505, 'loss_consonant': 0.151151, 'loss_word': 0.243136}
   22 | 0.000350 | 160000/160635 | 23.2937 | 17.4513 |
val: {'recall': 0.988243, 'recall_grapheme': 0.98287, 'recall_vowel': 0.993232, 'recall_consonant': 0.993999, 'recall_word': 0.982236, 'acc_grapheme': 0.981022, 'acc_vowel': 0.993011, 'acc_consonant': 0.993235, 'acc_word': 0.98239, 'loss_grapheme': 0.34441, 'loss_vowel': 0.244334, 'loss_consonant': 0.157811, 'loss_word': 0.282093}
   23 | 0.000346 | 160000/160635 | 16.0969 | 17.3355 |
val: {'recall': 0.987697, 'recall_grapheme': 0.983472, 'recall_vowel': 0.991819, 'recall_consonant': 0.992025, 'recall_word': 0.981513, 'acc_grapheme': 

###>>>>> saved
   42 | 0.000244 | 160000/160635 | 21.5886 | 16.1747 |
val: {'recall': 0.990179, 'recall_grapheme': 0.986513, 'recall_vowel': 0.993908, 'recall_consonant': 0.993783, 'recall_word': 0.986363, 'acc_grapheme': 0.984853, 'acc_vowel': 0.993583, 'acc_consonant': 0.994429, 'acc_word': 0.98637, 'loss_grapheme': 0.29636, 'loss_vowel': 0.225799, 'loss_consonant': 0.150296, 'loss_word': 0.187128}
   43 | 0.000238 | 160000/160635 | 0.9276 | 15.5587 ||
val: {'recall': 0.989957, 'recall_grapheme': 0.986283, 'recall_vowel': 0.993358, 'recall_consonant': 0.993906, 'recall_word': 0.985797, 'acc_grapheme': 0.985126, 'acc_vowel': 0.993856, 'acc_consonant': 0.994478, 'acc_word': 0.985897, 'loss_grapheme': 0.200493, 'loss_vowel': 0.146154, 'loss_consonant': 0.099368, 'loss_word': 0.133185}
   44 | 0.000231 | 160000/160635 | 0.9186 | 15.4935 ||
val: {'recall': 0.989522, 'recall_grapheme': 0.985582, 'recall_vowel': 0.993522, 'recall_consonant': 0.993404, 'recall_word': 0.986566, 'acc_grapheme'

###>>>>> saved
   63 | 0.000115 | 160000/160635 | 21.0869 | 15.4909 |
val: {'recall': 0.991303, 'recall_grapheme': 0.987797, 'recall_vowel': 0.994674, 'recall_consonant': 0.994944, 'recall_word': 0.988234, 'acc_grapheme': 0.987191, 'acc_vowel': 0.994652, 'acc_consonant': 0.99515, 'acc_word': 0.988285, 'loss_grapheme': 0.248291, 'loss_vowel': 0.195017, 'loss_consonant': 0.12395, 'loss_word': 0.15407}
   64 | 0.000109 | 160000/160635 | 17.6079 | 15.3241 |
val: {'recall': 0.991486, 'recall_grapheme': 0.987871, 'recall_vowel': 0.994552, 'recall_consonant': 0.995651, 'recall_word': 0.989058, 'acc_grapheme': 0.986818, 'acc_vowel': 0.994354, 'acc_consonant': 0.99505, 'acc_word': 0.989006, 'loss_grapheme': 0.301503, 'loss_vowel': 0.234748, 'loss_consonant': 0.153272, 'loss_word': 0.16906}
   65 | 0.000104 | 160000/160635 | 11.3319 | 15.2315 |
val: {'recall': 0.991995, 'recall_grapheme': 0.988611, 'recall_vowel': 0.995208, 'recall_consonant': 0.99555, 'recall_word': 0.988736, 'acc_grapheme': 0.

   84 | 0.000022 | 160000/160635 | 19.3180 | 14.6824 |
val: {'recall': 0.991856, 'recall_grapheme': 0.9882, 'recall_vowel': 0.99511, 'recall_consonant': 0.995914, 'recall_word': 0.989211, 'acc_grapheme': 0.987663, 'acc_vowel': 0.995001, 'acc_consonant': 0.995722, 'acc_word': 0.98923, 'loss_grapheme': 0.262447, 'loss_vowel': 0.205642, 'loss_consonant': 0.128688, 'loss_word': 0.160564}
   85 | 0.000019 | 160000/160635 | 16.4641 | 15.8382 |
val: {'recall': 0.99163, 'recall_grapheme': 0.988013, 'recall_vowel': 0.994678, 'recall_consonant': 0.995817, 'recall_word': 0.989342, 'acc_grapheme': 0.987191, 'acc_vowel': 0.994503, 'acc_consonant': 0.995399, 'acc_word': 0.989379, 'loss_grapheme': 0.314966, 'loss_vowel': 0.257951, 'loss_consonant': 0.16325, 'loss_word': 0.172734}
   86 | 0.000016 | 160000/160635 | 13.6084 | 15.3989 |
val: {'recall': 0.991518, 'recall_grapheme': 0.988468, 'recall_vowel': 0.994388, 'recall_consonant': 0.994748, 'recall_word': 0.989141, 'acc_grapheme': 0.987862, 'acc_vo

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc7c8eb40d0>
Traceback (most recent call last):
  File "/home/chec/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/chec/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [39]:
del model